In [1]:
import torchvision
import torchvision.transforms as transforms

In [2]:
import numpy as np
import torch

In [3]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt


In [4]:
# Define functions for integral image calculation
def integral_image(image):
    integral = np.cumsum(np.cumsum(image, axis=0), axis=1)
    return integral

In [5]:
def get_integral_sum(integral, x, y, w, h):
    a = integral[x, y]
    b = integral[x, y + h]
    c = integral[x + w, y]
    d = integral[x + w, y + h]
    return d - b - c + a

In [6]:
# Define the Haar-like features
def calculate_features(image):
    features = []
    height, width = image.shape
    for w in range(1, width + 1):
        for h in range(1, height + 1):
            for x in range(width - w + 1):
                for y in range(height - h + 1):
                    features.append(((x, y, w, h), 1))  # Type 1: Two-rectangle feature
                    if w % 2 == 0:
                        features.append(((x, y, w, h), 2))  # Type 2: Three-rectangle feature
                    if h % 2 == 0:
                        features.append(((x, y, w, h), 3))  # Type 3: Four-rectangle feature
    return features

In [7]:
# Train the weak classifiers
def train_weak_classifiers(features, images, labels):
    num_features = len(features)
    num_images = len(images)
    num_positives = sum(labels)
    num_negatives = num_images - num_positives
    pos_weight = 1.0 / (2 * num_positives)
    neg_weight = 1.0 / (2 * num_negatives)
    classifiers = []

    for feat_idx in range(num_features):
        feature, feature_type = features[feat_idx]
        if feature_type == 1:
            threshold = np.mean([get_integral_sum(integral_image(img), *feature) for img in images])
        else:
            threshold = -np.mean([get_integral_sum(integral_image(img), *feature) for img in images])
        error = sum([labels[i] * (get_integral_sum(integral_image(images[i]), *feature) <= threshold) for i in range(num_images)]) * pos_weight + sum([labels[i] * (get_integral_sum(integral_image(images[i]), *feature) > threshold) for i in range(num_images)]) * neg_weight
        alpha = 0.5 * np.log((1.0 - error) / max(error, 1e-16))
        classifiers.append((feature, threshold, alpha))

        # Update image weights
        predictions = np.array([get_integral_sum(integral_image(img), *feature) <= threshold for img in images])
        labels_float = np.array(labels) * 2.0 - 1.0
        weights = np.exp(-alpha * labels_float * predictions)
        weights /= np.sum(weights)
        labels = list((labels_float + 1.0) / 2.0)

    return classifiers

In [8]:
# Apply the strong classifier to an image
def apply_strong_classifier(image, classifiers):
    score = 0.0
    for feature, threshold, alpha in classifiers:
        if get_integral_sum(integral_image(image), *feature) <= threshold:
            score += alpha
        else:
            score -= alpha
    return score

In [9]:
'''


(X_train, y_train), (X_test,y_test) = datasets.cifar10.load_data()
X_train.shape

'''

'\n\n\n(X_train, y_train), (X_test,y_test) = datasets.cifar10.load_data()\nX_train.shape\n\n'

In [10]:
'''

plt.figure(figsize = (15,2))
plt.imshow(X_train[0])

'''


'\n\nplt.figure(figsize = (15,2))\nplt.imshow(X_train[0])\n\n'

In [11]:
'''


def load_cifar10():
    # Define the transformation to apply to the dataset
    transform = transforms.ToTensor()

    # Load the CIFAR-10 training dataset
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

    # Get the training images and labels
    train_images = []
    train_labels = []
    for images, labels in trainloader:
        train_images.append(images)
        train_labels.append(labels)
    train_images = torch.cat(train_images, dim=0)
    train_labels = torch.cat(train_labels, dim=0)

    return train_images, train_labels



'''

"\n\n\ndef load_cifar10():\n    # Define the transformation to apply to the dataset\n    transform = transforms.ToTensor()\n\n    # Load the CIFAR-10 training dataset\n    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)\n    trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)\n\n    # Get the training images and labels\n    train_images = []\n    train_labels = []\n    for images, labels in trainloader:\n        train_images.append(images)\n        train_labels.append(labels)\n    train_images = torch.cat(train_images, dim=0)\n    train_labels = torch.cat(train_labels, dim=0)\n\n    return train_images, train_labels\n\n\n\n"

In [12]:


def preprocess_cifar10(images):
    # Convert pixel values to floats between 0 and 1
    #preprocessed_images = images.float() / 255.0

    # Normalize the pixel values
    mean = torch.tensor([0.4914, 0.4822, 0.4465])  # Mean values for CIFAR-10 dataset
    std = torch.tensor([0.2023, 0.1994, 0.2010])  # Standard deviation values for CIFAR-10 dataset
    preprocessedd_images = (preprocessedd_images - mean[None, :, None, None]) / std[None, :, None, None]

    return preprocessedd_images


In [13]:
'''
# Preprocess CIFAR-10 dataset (example)
def preprocess_cifar10(images):
    # Preprocess CIFAR-10 dataset here
    # ...
    return preprocessed_images
'''

'\n# Preprocess CIFAR-10 dataset (example)\ndef preprocess_cifar10(images):\n    # Preprocess CIFAR-10 dataset here\n    # ...\n    return preprocessed_images\n'

In [14]:
# Train the Viola-Jones object detector
def train_viola_jones(images, labels, num_classifiers):
    preprocesseed_images = preprocess_cifar10(images)
    integral_images = [integral_image(img) for img in preprocesseed_images]
    features = calculate_features(preprocesseed_images[0])
    classifiers = train_weak_classifiers(features, integral_images, labels)
    sorted_classifiers = sorted(classifiers, key=lambda x: x[2], reverse=True)
    return sorted_classifiers[:num_classifiers]

In [15]:
# Test the Viola-Jones object detector
def test_viola_jones(images, classifiers):
    preprocessedddd_images = preprocess_cifar10(images)
    predictions = []
    for img in preprocessedddd_images:
        score = apply_strong_classifier(img, classifiers)
        predictions.append(score > 0.0)
    return predictions

In [16]:
# Example usage
#images, labels = load_cifar10()
(X_train, y_train), (X_test,y_test) = datasets.cifar10.load_data()
X_train = X_train / 255.0
X_test = X_test / 255.0

classifiers = train_viola_jones(X_train, y_train, num_classifiers=10)
#test_images = load_test_images()  # Load test images
predictions = test_viola_jones(X_test, classifiers)
print(predictions)

UnboundLocalError: local variable 'preprocessedd_images' referenced before assignment

In [ ]:
#train_images, train_labels, test_images, test_labels = load_cifar10()


In [ ]:
#preprocessed_test_images = preprocess_cifar10(test_images)


In [ ]:

'''

def load_cifar10():
    # Define the transformation to apply to the dataset
    transform = transforms.ToTensor()

    # Load the CIFAR-10 training dataset
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

    # Load the CIFAR-10 test dataset
    testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
    testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

    # Get the training images and labels
    train_images = []
    train_labels = []
    for images, labels in trainloader:
        train_images.append(images)
        train_labels.append(labels)
    train_images = torch.cat(train_images, dim=0)
    train_labels = torch.cat(train_labels, dim=0)

    # Get the test images and labels
    test_images = []
    test_labels = []
    for images, labels in testloader:
        test_images.append(images)
        test_labels.append(labels)
    test_images = torch.cat(test_images, dim=0)
    test_labels = torch.cat(test_labels, dim=0)

    return train_images, train_labels, test_images, test_labels


'''

